In [ ]:
%reload_ext autoreload
%autoreload 2

### Camera

In [ ]:
from datetime import datetime
from pathlib import Path

from inputs import get_gamepad
from IPython.display import display
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg
from jetracer.nvidia_racecar import NvidiaRacecar
import ipywidgets
import requests

In [ ]:
OUT_P = Path('train_dataset') # directorio de destino de las fotos (se creará)
FPS   = 1 # frames por segundo
SIZE  = 224

In [ ]:
moving = False

In [ ]:
def update_display():
    info = '\n'.join([
        f"th_gain: {car.throttle_gain:.2f}",
        f"st_offs: {car.steering_offset:.2f}",
    ])
    requests.get("http://localhost:8000/text/" + info)

In [ ]:
def reset_display():
    requests.get("http://localhost:8000/stats/on")

In [ ]:
def get_timestamp():
    return datetime.utcnow().strftime('%Y%m%d_%H%M%S_%f')

In [ ]:
def update_image(change):
    image = change['new']
    jpg = bgr8_to_jpeg(image)
    image_widget.value = jpg
    if car.throttle > 0.1 or car.throttle < -0.1:
        jpg_p = OUT_P / (get_timestamp() + ".jpg")
        with jpg_p.open('wb') as f:
            f.write(jpg)
        print(f"Th={car.throttle} - saving {jpg_p.name}.jpg")

In [ ]:
car = NvidiaRacecar()

In [ ]:
car.steering = 0
car.steering_offset = 0
car.throttle_gain = 0.20

In [ ]:
OUT_P.mkdir(exist_ok=True)

In [ ]:
camera = CSICamera(width=SIZE, height=SIZE, capture_fps=FPS)
camera.running = True

In [ ]:
image_widget = ipywidgets.Image(format='jpeg')

In [ ]:
camera.observe(update_image, names='value')

In [ ]:
display(image_widget)

In [ ]:
right_trigger = left_trigger = 0

while not (right_trigger and left_trigger):
    events = get_gamepad()
    for event in events:
        if event.ev_type == 'Absolute':
            if event.code == 'ABS_Y':
                car.throttle = -(event.state - 127.5) / 127.5
                #print(f'Y={event.state}')
            if event.code == 'ABS_Z':
                car.steering = (event.state - 127.5) / 127.5
                #print(f'Z={event.state}')
        elif event.ev_type == 'Key' and event.code == 'BTN_TR2':
            right_trigger = event.state
        elif event.ev_type == 'Key' and event.code == 'BTN_TL2':
            left_trigger = event.state
        elif event.ev_type == 'Key' and event.state == 1:
            if event.code == 'BTN_WEST': # Y / UP
                car.throttle_gain = min(1.0, car.throttle_gain + 0.05)
                update_display()
            if event.code == 'BTN_SOUTH': # A / DOWN
                car.throttle_gain = max(0.0, car.throttle_gain - 0.05)
                update_display()
            if event.code == 'BTN_EAST': # B / RIGHT
                car.steering_offset = max(-0.3, car.steering_offset - 0.05)
                car.steering = 0.1
                car.steering = 0
                update_display()
            if event.code == 'BTN_NORTH': # X / LEFT
                car.steering_offset = min(0.3, car.steering_offset + 0.05)
                car.steering = 0.1
                car.steering = 0
                update_display()

            #print(f"throttle_gain={car.throttle_gain:.1f}")
        #print(event.ev_type, event.code, event.state)

car.throttle = 0
reset_display()

In [ ]:
camera.unobserve(update_image, names='value')

In [ ]:
camera.running = False